In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# Paths
data_dir = r"D:\Final EEG Graph Project\EEG Images"  # your dataset root folder

# Data transforms (rescale to image size and normalize)
transform = transforms.Compose([
    transforms.Resize((2212, 1408)),  # Ensure all images are the same size
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

In [3]:
# Split dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=8)

In [4]:
# Flatten helper
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

# CNN Model using Sequential
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=2),  # -> (16, 1104, 702)
            nn.ReLU(),
            nn.MaxPool2d(2),                         # -> (16, 552, 351)

            nn.Conv2d(16, 32, kernel_size=3),           # -> (32, 550, 349)
            nn.LeakyReLU(0.01),
            nn.MaxPool2d(2),                         # -> (32, 275, 174)

            nn.Conv2d(32, 64, kernel_size=3),           # -> (64, 273, 172)
            nn.LeakyReLU(0.01),
            nn.MaxPool2d(2),                         # -> (64, 136, 86)

            nn.Conv2d(64, 128, kernel_size=3),          # -> (128, 136, 86)
            nn.ReLU(),
            nn.MaxPool2d(2),                         # -> (128, 68, 43)

            nn.Conv2d(128, 256, kernel_size=3),          # -> (256, 66, 41)
            nn.LeakyReLU(0.01),
            nn.MaxPool2d(2),                         # -> (256, 33, 20)

            nn.Conv2d(256, 512, kernel_size=3),          # -> (512, 66, 41)
            nn.LeakyReLU(0.01),
            nn.MaxPool2d(2),                         # -> (512, 33, 20)
            nn.AdaptiveAvgPool2d((7, 7)),

            Flatten(),
            nn.Linear(512 * 7 * 7, 1024),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.3),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Initialize model
model = CNN().to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [5]:
os.makedirs("models", exist_ok=True)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}")

    # Save model
    torch.save(model.state_dict(), f"D:/Final EEG Graph Project/models/model_state_dict/model_state_dict_epoch_{epoch+1}.pth")
    torch.save(model, f"D:/Final EEG Graph Project/models/full_model/full_model_epoch_{epoch+1}.pt")
    print(f"✅ Model saved: models/model_epoch_{epoch+1}.pth and models/full_model_epoch_{epoch+1}.pt")

Epoch 1/100, Loss: 68.6141
✅ Model saved: models/model_epoch_1.pth and models/full_model_epoch_1.pt
Epoch 2/100, Loss: 63.6055
✅ Model saved: models/model_epoch_2.pth and models/full_model_epoch_2.pt
Epoch 3/100, Loss: 52.5961
✅ Model saved: models/model_epoch_3.pth and models/full_model_epoch_3.pt
Epoch 4/100, Loss: 47.1886
✅ Model saved: models/model_epoch_4.pth and models/full_model_epoch_4.pt
Epoch 5/100, Loss: 47.9271
✅ Model saved: models/model_epoch_5.pth and models/full_model_epoch_5.pt
Epoch 6/100, Loss: 44.3620
✅ Model saved: models/model_epoch_6.pth and models/full_model_epoch_6.pt
Epoch 7/100, Loss: 43.9893
✅ Model saved: models/model_epoch_7.pth and models/full_model_epoch_7.pt
Epoch 8/100, Loss: 42.6662
✅ Model saved: models/model_epoch_8.pth and models/full_model_epoch_8.pt
Epoch 9/100, Loss: 41.8517
✅ Model saved: models/model_epoch_9.pth and models/full_model_epoch_9.pt
Epoch 10/100, Loss: 42.9485
✅ Model saved: models/model_epoch_10.pth and models/full_model_epoch_10.

In [9]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_acc = 0.0
best_epoch = None

for epoch in range(1, 101):
    model_path = fr"D:\Final EEG Graph Project\models\full_model\full_model_epoch_{epoch}.pt"
    try:
        model = torch.load(model_path, map_location=device)
        model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                preds = (outputs > 0.5).squeeze().long()
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        acc = 100 * correct / total
        print(f"Epoch {epoch}: Validation Accuracy = {acc:.2f}%")

        if acc > best_acc:
            best_acc = acc
            best_epoch = epoch

    except Exception as e:
        print(f"Failed to load model at epoch {epoch}: {e}")

print(f"\n✅ Best Model: Epoch {best_epoch} with Accuracy {best_acc:.2f}%")


C:\Users\suyes\AppData\Local\Temp\ipykernel_17284\664528689.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


Epoch 1: Validation Accuracy = 49.24%
Epoch 2: Validation Accuracy = 68.53%
Epoch 3: Validation Accuracy = 72.08%
Epoch 4: Validation Accuracy = 71.07%
Epoch 5: Validation Accuracy = 77.16%
Epoch 6: Validation Accuracy = 75.13%
Epoch 7: Validation Accuracy = 78.68%
Epoch 8: Validation Accuracy = 80.20%
Epoch 9: Validation Accuracy = 79.19%
Epoch 10: Validation Accuracy = 78.17%
Epoch 11: Validation Accuracy = 80.20%
Epoch 12: Validation Accuracy = 79.70%
Epoch 13: Validation Accuracy = 80.71%
Epoch 14: Validation Accuracy = 80.20%
Epoch 15: Validation Accuracy = 78.17%
Epoch 16: Validation Accuracy = 79.70%
Epoch 17: Validation Accuracy = 80.71%
Epoch 18: Validation Accuracy = 79.19%
Epoch 19: Validation Accuracy = 83.76%
Epoch 20: Validation Accuracy = 76.65%
Epoch 21: Validation Accuracy = 80.71%
Epoch 22: Validation Accuracy = 84.26%
Epoch 23: Validation Accuracy = 81.22%
Epoch 24: Validation Accuracy = 80.20%
Epoch 25: Validation Accuracy = 77.16%
Epoch 26: Validation Accuracy = 81

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# ✅ Choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load your trained model
model = torch.load(r"models/full_model/full_model_epoch_30.pt", map_location=device)
model.eval()

# ✅ Preprocessing for the input image
transform = transforms.Compose([
    transforms.Resize((2212, 1408)),  # Ensure all images are the same size
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# ✅ Load and preprocess the image
def predict_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)  # Add batch dimension

    # Predict
    with torch.no_grad():
        output = model(img_tensor)
        pred = (output > 0.5).item()

    # Show result
    label = "Normal" if pred == 1 else "Abnormal"
    print(f"Prediction: {label}")
    return label

# 🧪 Example usage
predict_image(r"D:\EEG graph project\EEG Images\normal graphs\P5_normal graphs_5.png")



C:\Users\suyes\AppData\Local\Temp\ipykernel_16484\2044414851.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(r"models/full_model/full_model_epoch_30.p

Prediction: Abnormal


'Abnormal'

In [13]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# ✅ Device selection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load the best model (Epoch 22 with 84.26% accuracy)
model_path = r"D:\Final EEG Graph Project\models\full_model\full_model_epoch_22.pt"
model = torch.load(model_path, map_location=device)
model.eval()

# ✅ Image preprocessing
transform = transforms.Compose([
    transforms.Resize((2212, 1408)),         # Ensure size matches training
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)    # For 3 RGB channels
])

# ✅ Prediction function with friendly output
def predict_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        pred = (output > 0.5).item()
        prob = output.item()

    if pred == 1:
        label = "Normal Brain Activity Detected.\n✅ No signs of abnormal neural patterns."
    else:
        label = ("⚠️ Abnormal Brain Activity Detected.\n"
                 "Please consult a neurologist for further evaluation.")

    print(f"Prediction Result:\n{label}\n(Confidence Score: {prob:.2f})")
    return label

# 🧪 Example usage
predict_image(r"D:\EEG graph project\EEG Images\abnormal graphs\P44_abnormal graphs_0.png")


C:\Users\suyes\AppData\Local\Temp\ipykernel_17284\3362797156.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


Prediction Result:
⚠️ Abnormal Brain Activity Detected.
Please consult a neurologist for further evaluation.
(Confidence Score: 0.20)


'⚠️ Abnormal Brain Activity Detected.\nPlease consult a neurologist for further evaluation.'